In [103]:
import torch
import sys
import yaml
from torchvision import transforms, datasets
import torchvision
import numpy as np
import os
from sklearn import preprocessing
from torch.utils.data.dataloader import DataLoader

In [104]:
sys.path.append('../')
from models.resnet_base_network import ResNet18

In [105]:
batch_size = 512
data_transforms = torchvision.transforms.Compose([transforms.Resize((256, 256)), 
                                                  transforms.ToTensor()])

In [106]:
config = yaml.load(open("../config/config.yaml", "r"), Loader=yaml.FullLoader)

In [107]:
def create_self_supervised_set(base_dir, target_data_name, self_train_num_per_class, self_test_num_per_class):
    ## 하나의 파일로 합치는 작업
    dataroot = '../../../../dataset'
    target_names = os.listdir(dataroot)
    print('>> check class : ', os.listdir(dataroot))
    
    file_train_names_list = list()
    file_test_names_list = list()
    
    for target_name in target_names : 
        target_path = os.path.join(dataroot, target_name)
        
        target_train_filenames = sorted(os.listdir(target_path))[:self_train_num_per_class]
        target_test_filenames = sorted(os.listdir(target_path))[self_train_num_per_class:self_train_num_per_class+self_test_num_per_class]
        
        for target_filename in target_train_filenames : 
            target_filepath = os.path.join(target_path, target_filename)
            file_train_names_list.append(target_filepath)
        for test_filename in target_test_filenames : 
            target_filepath = os.path.join(target_path, target_filename)
            file_test_names_list.append(target_filepath)

    # 저장할 파일 생성
    save_base = os.path.join(base_dir, target_data_name, 'train_size_' + str(self_train_num_per_class))
    train_save_dir_out = os.path.join(save_base, 'train/')
    train_save_dir = os.path.join(save_base, 'train/images/')
    
    if not os.path.exists(train_save_dir) :
        os.makedirs(train_save_dir)
    # train 파일 저장
    for src_filename in file_train_names_list:
        img = cv2.imread(src_filename)
        cv2.imwrite(os.path.join(train_save_dir, src_filename.split('/')[-1]), img)

    
    # 저장할 파일 생성
    test_save_dir_out = os.path.join(save_base, 'test/')
    test_save_dir = os.path.join(save_base, 'test/images/')
    
    if not os.path.exists(test_save_dir) :
        os.makedirs(test_save_dir)
    # test 파일 저장
    for src_filename in file_test_names_list:
        img = cv2.imread(src_filename)
        cv2.imwrite(os.path.join(test_save_dir, src_filename.split('/')[-1]), img)
    
    return train_save_dir_out, test_save_dir_out
    

In [122]:
#from utils import * 
train_dir, test_dir = create_self_supervised_set(base_dir = '../datasets/carclass_binary', target_data_name = 'ray_stonic' , self_train_num_per_class = 500, self_test_num_per_class = 100)

train_dataset = datasets.ImageFolder(root=train_dir, transform = data_transforms)
test_dataset = datasets.ImageFolder(root=test_dir, transform = data_transforms)

>> check class :  ['stonic', 'ray']


In [123]:
print("Input shape:", train_dataset[0][0].shape)
print("Input shape:", test_dataset[0][0].shape)

Input shape: torch.Size([3, 256, 256])
Input shape: torch.Size([3, 256, 256])


In [124]:
train_loader = DataLoader(train_dataset, batch_size=batch_size,
                          num_workers=0, drop_last=False, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=batch_size,
                          num_workers=0, drop_last=False, shuffle=True)

In [125]:
device = 'cpu' #'cuda' if torch.cuda.is_available() else 'cpu'
encoder = ResNet18(**config['network'])
output_feature_dim = encoder.projetion.net[0].in_features

In [126]:
#load pre-trained parameters
load_params = torch.load(os.path.join('../runs/Jul14_11-04-37_accida-esther-gpu-notebook1/checkpoints//model.pth'),
                         map_location=torch.device(torch.device(device)))

if 'online_network_state_dict' in load_params:
    encoder.load_state_dict(load_params['online_network_state_dict'])
    print("Parameters successfully loaded.")

# remove the projection head
encoder = torch.nn.Sequential(*list(encoder.children())[:-1])    
encoder = encoder.to(device)

Parameters successfully loaded.


In [127]:
encoder

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

In [128]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        return self.linear(x)

In [129]:
logreg = LogisticRegression(output_feature_dim, 10)
logreg = logreg.to(device)

In [130]:
def get_features_from_encoder(encoder, loader):
    
    x_train = []
    y_train = []

    # get the features from the pre-trained model
    for i, (x, y) in enumerate(loader):
        with torch.no_grad():
            feature_vector = encoder(x)
            x_train.extend(feature_vector)
            y_train.extend(y.numpy())

            
    x_train = torch.stack(x_train)
    y_train = torch.tensor(y_train)
    return x_train, y_train

In [131]:
encoder.eval()
x_train, y_train = get_features_from_encoder(encoder, train_loader)
x_test, y_test = get_features_from_encoder(encoder, test_loader)

if len(x_train.shape) > 2:
    x_train = torch.mean(x_train, dim=[2, 3])
    x_test = torch.mean(x_test, dim=[2, 3])
    
print("Training data shape:", x_train.shape, y_train.shape)
print("Testing data shape:", x_test.shape, y_test.shape)

Training data shape: torch.Size([1000, 512]) torch.Size([1000])
Testing data shape: torch.Size([204, 512]) torch.Size([204])


In [132]:
def create_data_loaders_from_arrays(X_train, y_train, X_test, y_test):

    train = torch.utils.data.TensorDataset(X_train, y_train)
    train_loader = torch.utils.data.DataLoader(train, batch_size=64, shuffle=True)

    test = torch.utils.data.TensorDataset(X_test, y_test)
    test_loader = torch.utils.data.DataLoader(test, batch_size=512, shuffle=False)
    return train_loader, test_loader

In [133]:
scaler = preprocessing.StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train).astype(np.float32)
x_test = scaler.transform(x_test).astype(np.float32)

In [134]:
train_loader, test_loader = create_data_loaders_from_arrays(torch.from_numpy(x_train), y_train, torch.from_numpy(x_test), y_test)

In [135]:
optimizer = torch.optim.Adam(logreg.parameters(), lr=3e-4)
criterion = torch.nn.CrossEntropyLoss()
eval_every_n_epochs = 10

for epoch in range(200):
#     train_acc = []
    for x, y in train_loader:

        x = x.to(device)
        y = y.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()        
        
        logits = logreg(x)
        predictions = torch.argmax(logits, dim=1)
        
        loss = criterion(logits, y)
        
        loss.backward()
        optimizer.step()
    
    total = 0
    if epoch % eval_every_n_epochs == 0:
        correct = 0
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)

            logits = logreg(x)
            predictions = torch.argmax(logits, dim=1)
            
            total += y.size(0)
            correct += (predictions == y).sum().item()
            
        acc = 100 * correct / total
        print(f"Testing accuracy: {np.mean(acc)}")

Testing accuracy: 12.745098039215685
Testing accuracy: 33.333333333333336
Testing accuracy: 56.86274509803921
Testing accuracy: 80.88235294117646
Testing accuracy: 90.68627450980392
Testing accuracy: 97.54901960784314
Testing accuracy: 100.0
Testing accuracy: 100.0
Testing accuracy: 100.0
Testing accuracy: 100.0
Testing accuracy: 100.0
Testing accuracy: 100.0
Testing accuracy: 100.0
Testing accuracy: 100.0
Testing accuracy: 100.0
Testing accuracy: 100.0
Testing accuracy: 100.0
Testing accuracy: 100.0
Testing accuracy: 100.0
Testing accuracy: 100.0
